In [1]:
import torch
import mlflow
import gc

from visualize_image import visualize_one_xy_slice_in_3d_image
from loading_dataset import load_data
from model import load_model
import numpy as np
from monai.data import box_utils
from monai.apps.detection.metrics.matching import matching_batch
from warmup_scheduler import GradualWarmupScheduler
from monai.apps.detection.metrics.coco import COCOMetric

In [2]:
import os

os.environ['MLFLOW_TRACKING_USERNAME'] = 'Eddy'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'Usp1#'
os.environ['MLFLOW_EXPERIMENT_NAME'] = 'LUNA16 + HC + MSD - without lung seg'
description = "Mixed dataset training without lung segmentation"

In [3]:
gt_box_mode = 'cccwhd'
batch_size = 8
patch_size = [96,96,40]
data_list_file_path = '/data/output/mixed_data/mixed_train_val0.json'
data_base_dir = '/data/mixed/'
# data_base_dir = '/data/HC_Images_resample/'
# data_base_dir = '/data/MSD_Images_resample/'
# data_base_dir = '/data/LUNA16_Images_resample/'
amp=True

returned_layers = [1,2]
base_anchor_shapes = [[6,8,4],[8,6,5],[10,10,6]]
conv1_t_stride = [2,2,1]
n_input_channels = 1
spatial_dims = 3
fg_labels = [0]
verbose = False
balanced_sampler_pos_fraction = 0.3
score_thresh = 0.02
nms_thresh = 0.22
val_patch_size = [256,256,104]

lr = 1e-2
val_interval = 5
coco_metric = COCOMetric(classes=["nodule"], iou_list=[0.1], max_detection=[100])
best_val_epoch_metric = 0.0
best_val_epoch = -1
max_epochs = 100
w_cls = 1.0

compute_dtype = torch.float32
if amp:
    compute_dtype = torch.float16

torch.backends.cudnn.benchmark = True

### loading dataset

In [4]:
train_loader, val_loader, len_train_ds = load_data(
    gt_box_mode, patch_size, batch_size, amp, data_list_file_path, data_base_dir
)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


### loading model

In [5]:
detector, device = load_model(
    returned_layers, base_anchor_shapes, conv1_t_stride, n_input_channels,
    spatial_dims, fg_labels, verbose, balanced_sampler_pos_fraction,
    score_thresh, nms_thresh, val_patch_size
)

### Initialize training

In [6]:
optimizer = torch.optim.SGD(
    detector.network.parameters(),
    lr,
    momentum=0.9,
    weight_decay=3e-5,
    nesterov=True
)

after_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=150, gamma=0.1
)
scheduler_warmup = GradualWarmupScheduler(
    optimizer, multiplier=1, total_epoch=10, after_scheduler=after_scheduler
)
scaler = torch.cuda.amp.GradScaler() if amp else None
optimizer.zero_grad()
optimizer.step()

### train

In [7]:
epoch_len = len_train_ds // train_loader.batch_size

with mlflow.start_run(description=description) as run:

    mlflow.log_param("gt_box_mode", gt_box_mode)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("patch_size", patch_size)
    mlflow.log_param("data_list_file_path", data_list_file_path)
    mlflow.log_param("data_base_dir", data_base_dir)
    mlflow.log_param("amp", amp)
    
    mlflow.log_param("n_input_channels", n_input_channels)
    mlflow.log_param("spatial_dims", spatial_dims)
    mlflow.log_param("balanced_sampler_pos_fraction", balanced_sampler_pos_fraction)
    mlflow.log_param("score_thresh", score_thresh)
    mlflow.log_param("nms_thresh", nms_thresh)
    
    mlflow.log_param("initial_lr", lr)
    mlflow.log_param("val_interval", val_interval)
    mlflow.log_param("max_epochs", max_epochs)
    mlflow.log_param("w_cls", w_cls)
    
    for epoch in range(max_epochs):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        detector.train()
        epoch_loss = 0
        epoch_cls_loss = 0
        epoch_box_reg_loss = 0
        step = 0
        scheduler_warmup.step()

        for batch_data in train_loader:
            step += 1
            inputs = [
                batch_data_ii["image"].to(device) for batch_data_i in batch_data for batch_data_ii in batch_data_i
            ]
            targets = [
                dict(
                    label=batch_data_ii["label"].to(device),
                    box=batch_data_ii["box"].to(device)
                )
                for batch_data_i in batch_data
                for batch_data_ii in batch_data_i
            ]

            for param in detector.network.parameters():
                param.grad = None

            if amp and (scaler is not None):
                with torch.cuda.amp.autocast():
                    outputs = detector(inputs, targets)
                    loss = w_cls * outputs[detector.cls_key] + outputs[detector.box_reg_key]
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = detector(inputs, targets)
                loss = w_cls * outputs[detector.cls_key] + outputs[detector.box_reg_key]
                loss.backward()
                optimizer.step()

            # saving into mlflow
            epoch_loss += loss.detach().item()
            epoch_cls_loss += outputs[detector.cls_key].detach().item()
            epoch_box_reg_loss += outputs[detector.box_reg_key].detach().item()
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            mlflow.log_metric("train_loss", loss.detach().item(), epoch_len * epoch + step)

        del inputs, batch_data
        torch.cuda.empty_cache()
        gc.collect()

        epoch_loss /= step
        epoch_cls_loss /= step
        epoch_box_reg_loss /= step
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        mlflow.log_metric("avg_train_loss", epoch_loss, epoch + 1)
        mlflow.log_metric("avg_train_cls_loss", epoch_cls_loss, epoch + 1)
        mlflow.log_metric("avg_train_box_reg_loss", epoch_box_reg_loss, epoch + 1)
        mlflow.log_metric("train_lr", optimizer.param_groups[0]["lr"], epoch + 1)

        # saving last trained model
        mlflow.pytorch.log_model(detector.network, "model")

        # validation for model selection
        if (epoch + 1) % val_interval == 0:
            detector.eval()
            val_outputs_all = []
            val_targets_all = []

            with torch.no_grad():
                for val_data in val_loader:
                    # if all val_data_i["image"] smaller than val_patch_size, no need to use inferer
                    # otherwise, need inferer to handle large input images.
                    use_inferer = not all(
                        [val_data_i["image"][0, ...].numel() < np.prod(val_patch_size) for val_data_i in val_data]
                    )
                    val_inputs = [val_data_i.pop("image").to(device) for val_data_i in val_data]

                    if amp:
                        with torch.cuda.amp.autocast():
                            val_outputs = detector(val_inputs, use_inferer=use_inferer)
                    else:
                        val_outputs = detector(val_inputs, use_inferer=use_inferer)

                    # save outputs for evaluation
                    val_outputs_all += val_outputs
                    val_targets_all += val_data

            # visualize an inference image and boxes
            print(val_data[0]["image_meta_dict"]["filename_or_obj"])
            draw_img = visualize_one_xy_slice_in_3d_image(
                gt_boxes=val_data[0]["box"].cpu().detach().numpy(),
                image=val_inputs[0][0, ...].cpu().detach().numpy(),
                pred_boxes=val_outputs[0][detector.target_box_key].cpu().detach().numpy(),
            )
            # mlflow.log_image(draw_img.transpose([2, 1, 0]), "val_img_xy.png")
            mlflow.log_image(draw_img, str(epoch + 1) + "_val_img_xy.png")

            # compute metrics
            del val_inputs
            torch.cuda.empty_cache()
            results_metric = matching_batch(
                iou_fn=box_utils.box_iou,
                iou_thresholds=coco_metric.iou_thresholds,
                pred_boxes=[
                    val_data_i[detector.target_box_key].cpu().detach().numpy() for val_data_i in val_outputs_all
                ],
                pred_classes=[
                    val_data_i[detector.target_label_key].cpu().detach().numpy() for val_data_i in val_outputs_all
                ],
                pred_scores=[
                    val_data_i[detector.pred_score_key].cpu().detach().numpy() for val_data_i in val_outputs_all
                ],
                gt_boxes=[val_data_i[detector.target_box_key].cpu().detach().numpy() for val_data_i in val_targets_all],
                gt_classes=[
                    val_data_i[detector.target_label_key].cpu().detach().numpy() for val_data_i in val_targets_all
                ]
            )
            val_epoch_metric_dict = coco_metric(results_metric)[0]
            print(val_epoch_metric_dict)

            # write metrics
            for k in val_epoch_metric_dict.keys():
                mlflow.log_metric("val_" + k, val_epoch_metric_dict[k], epoch + 1)
            val_epoch_metric = val_epoch_metric_dict.values()
            val_epoch_metric = sum(val_epoch_metric) / len(val_epoch_metric)
            mlflow.log_metric("val_metric", val_epoch_metric, epoch + 1)

            # save best trained model
            if val_epoch_metric > best_val_epoch_metric:
                best_val_epoch_metric = val_epoch_metric
                best_val_epoch = epoch + 1
                mlflow.pytorch.log_model(detector.network, "best_model")
            print(
                "current epoch: {} current metric: {:.4f} "
                "best metric: {:.4f} at epoch {}".format(
                    epoch + 1, val_epoch_metric, best_val_epoch_metric, best_val_epoch
                )
            )

    print(f"train completed, best_metric: {best_val_epoch_metric:.4f} " f"at epoch: {best_val_epoch}")

----------
epoch 1/100
1/463, train_loss: 1.0020
2/463, train_loss: 1.1660
3/463, train_loss: 1.3271
4/463, train_loss: 1.1895
5/463, train_loss: 1.1875
6/463, train_loss: 1.0645
7/463, train_loss: 0.8818
8/463, train_loss: 0.7129
9/463, train_loss: 1.4824
10/463, train_loss: 0.9961
11/463, train_loss: 1.0508
12/463, train_loss: 1.1592
13/463, train_loss: 0.8228
14/463, train_loss: 0.6079
15/463, train_loss: 0.7148
16/463, train_loss: 1.1650
17/463, train_loss: 0.5742
18/463, train_loss: 1.0312
19/463, train_loss: 0.7124
20/463, train_loss: 0.7246
21/463, train_loss: 0.6182
22/463, train_loss: 0.8145
23/463, train_loss: 1.1562
24/463, train_loss: 0.9102
25/463, train_loss: 0.9253
26/463, train_loss: 0.8535
27/463, train_loss: 0.7021
28/463, train_loss: 0.9316
29/463, train_loss: 0.7158
30/463, train_loss: 0.9092
31/463, train_loss: 0.9204
32/463, train_loss: 0.4470
33/463, train_loss: 0.5957
34/463, train_loss: 0.6455
35/463, train_loss: 1.0957
36/463, train_loss: 1.1113
37/463, train_

2023/09/20 06:01:07 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/20 06:01:11 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


----------
epoch 2/100
1/463, train_loss: 0.5210
2/463, train_loss: 0.5825
3/463, train_loss: 0.7500
4/463, train_loss: 0.6016
5/463, train_loss: 0.7520
6/463, train_loss: 0.6543
7/463, train_loss: 0.5947
8/463, train_loss: 0.5342
9/463, train_loss: 0.7490
10/463, train_loss: 0.6055
11/463, train_loss: 0.6465
12/463, train_loss: 0.5337
13/463, train_loss: 0.6865
14/463, train_loss: 0.4043
15/463, train_loss: 0.7573
16/463, train_loss: 0.6128
17/463, train_loss: 0.5449
18/463, train_loss: 0.5933
19/463, train_loss: 0.6377
20/463, train_loss: 0.4551
21/463, train_loss: 0.7363
22/463, train_loss: 0.5278
23/463, train_loss: 0.7686
24/463, train_loss: 0.5542
25/463, train_loss: 0.4312
26/463, train_loss: 0.6133
27/463, train_loss: 0.5806
28/463, train_loss: 0.5771
29/463, train_loss: 0.3804
30/463, train_loss: 0.7607
31/463, train_loss: 0.7729
32/463, train_loss: 0.6050
33/463, train_loss: 0.8184
34/463, train_loss: 0.6616
35/463, train_loss: 0.4937
36/463, train_loss: 0.5386
37/463, train_

2023/09/20 08:14:47 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/20 08:14:50 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


----------
epoch 3/100
1/463, train_loss: 0.6772
2/463, train_loss: 0.3438
3/463, train_loss: 0.2708
4/463, train_loss: 0.2263
5/463, train_loss: 0.4644
6/463, train_loss: 0.3750
7/463, train_loss: 0.7588
8/463, train_loss: 0.2141
9/463, train_loss: 0.2717
10/463, train_loss: 0.4111
11/463, train_loss: 0.4399
12/463, train_loss: 0.6338
13/463, train_loss: 0.7871
14/463, train_loss: 0.4341
15/463, train_loss: 0.5020
16/463, train_loss: 0.4668
17/463, train_loss: 0.2776
18/463, train_loss: 0.7881
19/463, train_loss: 0.4004
20/463, train_loss: 0.4614
21/463, train_loss: 0.5132
22/463, train_loss: 0.3274
23/463, train_loss: 0.5581
24/463, train_loss: 0.5605
25/463, train_loss: 0.6201
26/463, train_loss: 0.3008
27/463, train_loss: 0.5298
28/463, train_loss: 0.4277
29/463, train_loss: 0.6938
30/463, train_loss: 0.4531
31/463, train_loss: 0.8223
32/463, train_loss: 0.2412
33/463, train_loss: 0.6216
34/463, train_loss: 0.6064
35/463, train_loss: 0.3394
36/463, train_loss: 0.3594
37/463, train_

...

2023/09/22 22:30:12 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/22 22:30:15 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/22 22:30:18 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /mlflow/5/93273c7eb040429a9470b98ebd6ffb12/artifacts. Set logging level to DEBUG via 

----------
epoch 30/100
1/463, train_loss: 0.0914
2/463, train_loss: 0.1387
3/463, train_loss: 0.1440
4/463, train_loss: 0.1859
5/463, train_loss: 0.1165
6/463, train_loss: 0.2610
7/463, train_loss: 0.2808
8/463, train_loss: 0.0723
9/463, train_loss: 0.4216
10/463, train_loss: 0.1333
11/463, train_loss: 0.1132
12/463, train_loss: 0.1494
13/463, train_loss: 0.0878
14/463, train_loss: 0.3979
15/463, train_loss: 0.1168
16/463, train_loss: 0.0753
17/463, train_loss: 0.1714
18/463, train_loss: 0.1791
19/463, train_loss: 0.1755
20/463, train_loss: 0.3066
21/463, train_loss: 0.0911
22/463, train_loss: 0.3125
23/463, train_loss: 0.1131
24/463, train_loss: 0.2460
25/463, train_loss: 0.0552
26/463, train_loss: 0.0920
27/463, train_loss: 0.2019
28/463, train_loss: 0.1061
29/463, train_loss: 0.5332
30/463, train_loss: 0.2039
31/463, train_loss: 0.3535
32/463, train_loss: 0.2485
33/463, train_loss: 0.0919
34/463, train_loss: 0.0657
35/463, train_loss: 0.0879
36/463, train_loss: 0.1609
37/463, train

2023/09/23 00:43:28 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/23 00:43:30 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/09/23 00:43:33 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /mlflow/5/93273c7eb040429a9470b98ebd6ffb12/artifacts. Set logging level to DEBUG via 

/data/mixed/1.3.6.1.4.1.14519.5.2.1.6279.6001.153536305742006952753134773630/1.3.6.1.4.1.14519.5.2.1.6279.6001.153536305742006952753134773630.nii.gz
{'mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.5246881924598294, 'nodule_mAP_IoU_0.10_0.50_0.05_MaxDet_100': 0.5246881924598294, 'AP_IoU_0.10_MaxDet_100': 0.5431999499623729, 'nodule_AP_IoU_0.10_MaxDet_100': 0.5431999499623729, 'mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.9145299196243286, 'nodule_mAR_IoU_0.10_0.50_0.05_MaxDet_100': 0.9145299196243286, 'AR_IoU_0.10_MaxDet_100': 0.9487179517745972, 'nodule_AR_IoU_0.10_MaxDet_100': 0.9487179517745972}
current epoch: 30 current metric: 0.7328 best metric: 0.7468 at epoch 15
----------
epoch 31/100
1/463, train_loss: 0.2045
2/463, train_loss: 0.3049
3/463, train_loss: 0.1057
4/463, train_loss: 0.2588
5/463, train_loss: 0.1213
6/463, train_loss: 0.1765
7/463, train_loss: 0.0973
8/463, train_loss: 0.1333
9/463, train_loss: 0.3308
10/463, train_loss: 0.0224
11/463, train_loss: 0.0906
12/463, train_loss: 0.117

KeyboardInterrupt: 